In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../input/bank-train.csv', index_col = 0)

In [3]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [4]:
import sys
sys.path.append('../src/')

In [5]:
from feature_selection import Feature_Selection

In [6]:
df['target'] = df['y']
df.drop(['y'], axis = 1, inplace = True)

In [7]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
df['target'] = lbl.fit_transform(df['target'].values)


In [8]:
s = Feature_Selection(df, selector = 'vif', vif_threshold = 0.2, mixed = False)

In [9]:
s.get_features()

vif


['age',
 'pdays',
 'balance',
 'day',
 'campaign',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [10]:
s = Feature_Selection(df, selector = 'hclust')

In [ ]:
s.get_features()

hclust


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
